In [ ]:
# Copyright 2023 Google LLC
#
# Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
#     https://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

使用Vertex AI远程培训和Vertex AI SDK 2.0训练一个TabNet模型

<table align="left">
  <td>
    <a href="https://colab.research.google.com/github/GoogleCloudPlatform/vertex-ai-samples/blob/main/notebooks/official/tabnet/sdk2_remote_tabnet_training.ipynb">
      <img src="https://cloud.google.com/ml-engine/images/colab-logo-32px.png" alt="Colab logo"> 在Colab上运行
    </a>
  </td>
  <td>
    <a href="https://github.com/GoogleCloudPlatform/vertex-ai-samples/blob/main/notebooks/official/tabnet/sdk2_remote_tabnet_training.ipynb">
        <img src="https://cloud.google.com/ml-engine/images/github-logo-32px.png" alt="GitHub logo">
      在GitHub上查看
    </a>
  </td>
    <td>
    <a href="https://console.cloud.google.com/vertex-ai/workbench/deploy-notebook?download_url=https://raw.githubusercontent.com/GoogleCloudPlatform/vertex-ai-samples/main/notebooks/official/tabnet/sdk2_remote_tabnet_training.ipynb">
       <img src="https://www.gstatic.com/cloud/images/navigation/vertex-ai.svg" alt="Vertex AI logo">在Vertex AI工作台中打开
    </a>
</table>

## 概观

本教程演示了如何使用Vertex AI SDK 2.0对本地TabNet模型训练任务进行远程TabNet模型训练。

### 目标

在本教程中，您学习如何使用`Vertex AI SDK 2.0`来远程训练一个 TabNet 模型作为一个本地（on-prem）的训练任务。

本教程使用以下 Google Cloud ML 服务：

- `Vertex AI 训练`
- `Vertex AI 远程训练`

执行的步骤包括：

- 下载并拆分数据集
- 将数据导入 Dataframe 并进行转换。
- 训练一个表格分类模型。
- 训练一个表格回归模型。

**TabNet 远程训练**
```
from google.cloud.aiplatform.private_preview.vertex_ai.tabular_models import TabNetTrainer

vertex_ai.init(remote=True, project="my-project", location="my-location", staging_bucket="gs://my-bucket")

# 实例化 TabNetTrainer
tabnet_trainer = TabNetTrainer(...)

# 可选设置训练配置
tabnet_trainer.fit.vertex.training_config.staging_bucket = "gs://my-bucket"

# 这个 `fit` 调用将被远程执行
tabnet_trainer.fit(...)

# 这个 `predict` 调用将在本地执行
tabnet_trainer.predict(...)
```

数据集

本教程使用<a href="https://scikit-learn.org/stable/auto_examples/datasets/plot_iris_dataset.html">IRIS数据集</a>，用于预测鸢尾花的种类。

### 成本

本教程使用谷歌云的计费组件：

* Vertex AI
* 云存储

了解[Vertex AI
价格](https://cloud.google.com/vertex-ai/pricing)、[云存储
价格](https://cloud.google.com/storage/pricing)，并使用[Pricing
Calculator](https://cloud.google.com/products/calculator/)
根据您的预期使用量生成成本估算。

## 安装

安装以下包，以执行此笔记本所需的操作。

In [ ]:
! pip3 install --upgrade --quiet google-cloud-aiplatform[preview] \
                                 scikit-learn \
                                 pyarrow \
                                 fastparquet \
                                 tensorflow

仅限CoLab使用：取消下面的单元格注释以重新启动内核

In [ ]:
# Automatically restart kernel after installs so that your environment can access the new packages
# import IPython

# app = IPython.Application.instance()
# app.kernel.do_shutdown(True)

## 在开始之前

### 设置您的项目ID

**如果您不知道您的项目ID**，请尝试以下操作：
* 运行 `gcloud config list`。
* 运行 `gcloud projects list`。
* 参考支持页面：[查找项目ID](https://support.google.com/googleapi/answer/7014113)。

In [ ]:
PROJECT_ID = "[your-project-id]"  # @param {type:"string"}

# Set the project id
! gcloud config set project {PROJECT_ID}

#### 区域

您还可以更改 Vertex AI 使用的 `REGION` 变量。了解有关 [Vertex AI 区域](https://cloud.google.com/vertex-ai/docs/general/locations)的更多信息。

In [ ]:
REGION = "us-central1"

### 验证您的Google Cloud帐户

根据您的Jupyter环境，您可能需要手动进行身份验证。请按照以下相关说明操作。

**1. Vertex AI工作台**
* 无需操作，因为您已经完成了身份验证。

**2. 本地JupyterLab实例，请取消注释并运行：**

In [ ]:
# ! gcloud auth login

3. 协作，取消注释并运行：

In [ ]:
# from google.colab import auth
# auth.authenticate_user()

4. 服务账户或其他
* 查看如何将 Cloud Storage 权限授予您的服务账户：https://cloud.google.com/storage/docs/gsutil/commands/iam#ch-examples。

创建一个云存储桶

创建一个存储桶来存储中间产物，例如数据集。

In [ ]:
BUCKET_URI = f"gs://your-bucket-name-{PROJECT_ID}-unique"  # @param {type:"string"}

只有当您的存储桶尚不存在时：运行以下单元格来创建您的云存储存储桶。

In [ ]:
! gsutil mb -l {REGION} -p {PROJECT_ID} {BUCKET_URI}

### 设置变量

接下来，设置一些在教程中使用的变量。
### 导入库并定义常量

In [ ]:
import uuid

import pandas as pd
import vertexai.preview
from sklearn.datasets import load_iris
from sklearn.model_selection import train_test_split
from vertexai.preview.tabular_models import TabNetTrainer

初始化Python的Vertex AI SDK

为您的项目和相应的存储桶初始化Python的Vertex AI SDK。

In [ ]:
vertexai.init(
    project=PROJECT_ID,
    location=REGION,
    staging_bucket=BUCKET_URI,
)

### 准备数据集

现在加载鸢尾花数据集，并将数据分为训练、评估和测试集。

In [ ]:
dataset = load_iris()

X, X_validation, y, y_validation = train_test_split(
    dataset.data, dataset.target, test_size=0.60, random_state=42
)
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.20, random_state=42
)

print("Data size: ", len(dataset.target))
print("X_train size: ", len(X_train))
print("X_validation size: ", len(X_validation))
print("X_test size: ", len(X_test))

特征转换

接下来，您要将训练和评估数据导入pandas Dataframe中，并进行特征转换。

In [ ]:
columns = [f"arg_{i}" for i in range(X_train.shape[1])]

training_data = pd.DataFrame(X_train, columns=columns)
training_data["target"] = y_train
training_data["target"] = training_data["target"].astype("category")

validation_data = pd.DataFrame(X_validation, columns=columns)
validation_data["target"] = y_validation
validation_data["target"] = validation_data["target"].astype("category")

eval_data = pd.DataFrame(X_test, columns=columns)

training_data.head()

## 表格分类模型

初始化TabNetTrainer

接下来，初始化TabNetTrainer用于训练一个表格分类模型。

In [ ]:
tabnet_trainer = TabNetTrainer(
    model_type="classification",
    target_column="target",
    learning_rate=0.01,
    max_train_secs=1800,
)

远程训练

现在，将一个TabNet表格分类模型作为远程训练工作。

In [ ]:
REMOTE_JOB_NAME = "test-sdk2-remote-tabnet-training"
REMOTE_JOB_BUCKET = "/".join([BUCKET_URI, REMOTE_JOB_NAME])

# Sets the staging bucket for remote training
tabnet_trainer.fit.vertex.remote_config.staging_bucket = (
    REMOTE_JOB_BUCKET + f"_{uuid.uuid4()}"
)

# Executes remote training
tabnet_trainer.fit(training_data=training_data, validation_data=validation_data)

最后，使用训练好的表格分类模型进行本地预测。

In [ ]:
from sklearn.metrics import accuracy_score

predictions = tabnet_trainer.predict(eval_data)
accuracy_score(y_test, predictions)

## 表格回归模型

初始化TabNetTrainer

接下来，初始化TabNetTrainer用于训练一个表格回归模型。

In [ ]:
tabnet_trainer = TabNetTrainer(
    model_type="regression",
    target_column="target",
    learning_rate=0.01,
    max_train_secs=1800,
)

远程训练

现在，将一个TabNet表格回归模型作为远程训练任务。

In [ ]:
# Sets the staging bucket for remote training
tabnet_trainer.fit.vertex.remote_config.staging_bucket = (
    REMOTE_JOB_BUCKET + f"_{uuid.uuid4()}"
)

# Executes remote training
tabnet_trainer.fit(training_data=training_data, validation_data=validation_data)

### 本地预测

最后，使用训练好的表格回归模型进行本地预测。

In [ ]:
from sklearn.metrics import mean_squared_error

predictions = tabnet_trainer.predict(eval_data)
mean_squared_error(y_test, predictions, squared=False)

清理

要清理此项目中使用的所有Google Cloud资源，您可以[删除用于本教程的Google Cloud项目](https://cloud.google.com/resource-manager/docs/creating-managing-projects#shutting_down_projects)。

否则，您可以删除在本教程中创建的各个资源。

In [ ]:
import os

delete_bucket = False

if delete_bucket or os.getenv("IS_TESTING"):
    ! gsutil rm -rf {BUCKET_URI}